In [102]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from datetime import time
import matplotlib.pyplot as pplot
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
import math
import random
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.cross_validation import ShuffleSplit, train_test_split
from sklearn.metrics import mean_squared_error
path_final = 'dados_finais/'
from sklearn import preprocessing
from sklearn import svm

In [103]:
df_test1 = pd.read_csv(path_final+"test1.csv")
df_test2 = pd.read_csv(path_final+"test2.csv")
df_test3 = pd.read_csv(path_final+"test3.csv")
df_test4 = pd.read_csv(path_final+"test4.csv")
df_train1 = pd.read_csv(path_final+"train1.csv")
df_train2 = pd.read_csv(path_final+"train2.csv")
df_train3 = pd.read_csv(path_final+"train3.csv")
df_train4 = pd.read_csv(path_final+"train4.csv")

In [104]:
def adiciona_media_desvio_por_dia(df):
    df['media_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])["volume"].transform(np.mean)
    df['desvio_padrao_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])["volume"].transform(np.std)
    df['desvio_padrao_hora_dia'].fillna(df.groupby(['direction', 'tollgate_id', 'day', 'hour'])["volume"].transform(np.mean), inplace=True)
    df['min_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])['volume'].transform(np.min)
    df['max_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])['volume'].transform(np.max)
    df['mediana_volume_hora_dia'] = df.groupby(['direction', 'tollgate_id', 'day', 'hour'])['volume'].transform(np.median)
    return df

In [105]:
df_train1 = adiciona_media_desvio_por_dia(df_train1)
df_train2 = adiciona_media_desvio_por_dia(df_train2)
df_train3 = adiciona_media_desvio_por_dia(df_train3)
df_train4 = adiciona_media_desvio_por_dia(df_train4)
df_test1 = adiciona_media_desvio_por_dia(df_test1)

In [106]:
df_train1.head()

,time_start,volume,direction,tollgate_id,time_window,date,weekday,weekend,hour,minute,...,am_pm,volume_anterior,volume_anterior_2,media_volume,desvio_padrao,media_volume_hora_dia,desvio_padrao_hora_dia,min_volume_hora_dia,max_volume_hora_dia,mediana_volume_hora_dia
0,2016-09-19 00:00:00,13,0,1,"[00:00:00,00:20:00)",2016-09-19,1,0,0,0,...,1,0.0,0.0,11.4,2.190890,9.833333,2.316607,6,13,10.0
1,2016-09-19 00:20:00,6,0,1,"[00:20:00,00:40:00)",2016-09-19,1,0,0,20,...,1,13.0,0.0,10.0,4.242641,9.833333,2.316607,6,13,10.0
2,2016-09-19 00:40:00,9,0,1,"[00:40:00,01:00:00)",2016-09-19,1,0,0,40,...,1,6.0,13.0,8.4,2.966479,9.833333,2.316607,6,13,10.0
3,2016-09-19 01:00:00,10,0,1,"[01:00:00,01:20:00)",2016-09-19,1,0,1,0,...,1,9.0,6.0,5.8,3.033150,10.000000,3.162278,6,14,10.0
4,2016-09-19 01:20:00,14,0,1,"[01:20:00,01:40:00)",2016-09-19,1,0,1,20,...,1,10.0,9.0,7.2,5.069517,10.000000,3.162278,6,14,10.0


In [107]:
df_train_list = [df_train1,df_train2, df_train3, df_train4]
df_test_list = [df_test1, df_test2, df_test3, df_test4]

In [108]:
random.shuffle(df_train_list)
df_train = pd.concat(df_train_list)

/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


In [109]:
df_train[(df_train['date'] == '2016-10-10') & (df_train['hour'] == 8)].head()

,am_pm,date,day,desvio_padrao,desvio_padrao_hora_dia,direction,hour,max_volume_hora_dia,media_volume,media_volume_hora_dia,...,min_volume_hora_dia,minute,time_start,time_window,tollgate_id,volume,volume_anterior,volume_anterior_2,weekday,weekend
1032,1,2016-10-10,10,9.000000,3.511885,0,8,53,49.0,49.666667,...,46,NaN,2016-10-10 08:10:00,"[08:10:00,08:30:00)",1,46,43.0,39.0,1,0
1033,1,2016-10-10,10,4.816638,3.511885,0,8,53,50.2,49.666667,...,46,NaN,2016-10-10 08:30:00,"[08:30:00,08:50:00)",1,53,46.0,43.0,1,0
1034,1,2016-10-10,10,5.019960,3.511885,0,8,53,48.8,49.666667,...,46,NaN,2016-10-10 08:50:00,"[08:50:00,09:10:00)",1,50,53.0,46.0,1,0
3119,1,2016-10-10,10,12.198361,4.041452,0,8,132,127.6,127.666667,...,124,NaN,2016-10-10 08:10:00,"[08:10:00,08:30:00)",2,127,123.0,107.0,1,0
3120,1,2016-10-10,10,15.658863,4.041452,0,8,132,121.2,127.666667,...,124,NaN,2016-10-10 08:30:00,"[08:30:00,08:50:00)",2,132,127.0,123.0,1,0


In [110]:
df_test = df_test1

In [111]:
df_remove = df_test.loc[(df_test['volume'] == 0)]

df_test = df_test.drop(df_remove.index)

In [112]:
df_test.head()

,time_start,volume,direction,tollgate_id,time_window,date,weekday,weekend,hour,day,am_pm,volume_anterior,volume_anterior_2,media_volume,desvio_padrao,media_volume_hora_dia,desvio_padrao_hora_dia,min_volume_hora_dia,max_volume_hora_dia,mediana_volume_hora_dia
18,2016-10-25 06:00:00,10,0,1,"[06:00:00,06:20:00)",2016-10-25,2,0,6,25,1,0.0,0.0,14.285714,6.447591,19.000000,10.148892,10,30,17
19,2016-10-25 06:20:00,17,0,1,"[06:20:00,06:40:00)",2016-10-25,2,0,6,25,1,10.0,0.0,22.428571,14.808299,19.000000,10.148892,10,30,17
20,2016-10-25 06:40:00,30,0,1,"[06:40:00,07:00:00)",2016-10-25,2,0,6,25,1,17.0,10.0,28.714286,14.941154,19.000000,10.148892,10,30,17
21,2016-10-25 07:00:00,25,0,1,"[07:00:00,07:20:00)",2016-10-25,2,0,7,25,1,30.0,17.0,39.285714,26.208868,35.666667,9.712535,25,44,38
22,2016-10-25 07:20:00,38,0,1,"[07:20:00,07:40:00)",2016-10-25,2,0,7,25,1,25.0,30.0,45.428571,23.308490,35.666667,9.712535,25,44,38


In [113]:
def feature_format():
    #pd_volume_train = pd_volume_train.set_index(['time'])
    #pd_volume_test = pd_volume_test.set_index(['time'])
    #volume_train = v_train.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #volume_test = v_test.groupby(['time_window','tollgate_id','direction','date', 'hour']).size().reset_index().rename(columns = {0:'volume'})
    #print(volume_train)                
    x = pd.Series(df_train['time_window'].unique())
    s = pd.Series(range(len(x)),index = x.values)
    df_train['window_n'] = df_train['time_window'].map(s)
    df_test['window_n'] = df_test['time_window'].map(s)
#        print vol_test.tail()
    #volume_train['weekday'] = v_train['weekday']
    #volume_test['weekday'] = v_test['weekday']
    
    feature_train = df_train.drop('volume', axis = 1)
    feature_test = df_test.drop('volume',axis = 1)
    values_train = df_train['volume'].values
    values_test = df_test['volume'].values
    
    return feature_train, feature_test, values_train, values_test

In [123]:
feature_train, feature_test, values_train, values_test = feature_format()
feature_train = pd.concat([feature_train, pd.get_dummies(feature_train['tollgate_id'])], axis=1)
feature_test = pd.concat([feature_test, pd.get_dummies(feature_test['tollgate_id'])], axis=1)

In [118]:
feature_train.columns

Index([                  'am_pm',                    'date',
                           'day',           'desvio_padrao',
        'desvio_padrao_hora_dia',               'direction',
                          'hour',     'max_volume_hora_dia',
                  'media_volume',   'media_volume_hora_dia',
       'mediana_volume_hora_dia',     'min_volume_hora_dia',
                        'minute',              'time_start',
                   'time_window',             'tollgate_id',
               'volume_anterior',       'volume_anterior_2',
                       'weekday',                 'weekend',
                      'window_n',                         1,
                               2,                         3],
      dtype='object')

In [143]:
regressor_cubic = RandomForestRegressor(n_estimators = 1500, max_depth=10, max_features=None)

In [157]:
regressor_cubic.fit(feature_train[['tollgate_id','direction', 'hour','weekday', 'weekend','volume_anterior', 'volume_anterior_2', 'desvio_padrao', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia']], values_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [158]:
y_pred = regressor_cubic.predict(feature_test[[1,2,3,'direction', 'hour','weekday', 'weekend','volume_anterior', 'volume_anterior_2', 'desvio_padrao', 'am_pm', 'window_n', 'desvio_padrao_hora_dia', 'mediana_volume_hora_dia', 'max_volume_hora_dia', 'media_volume_hora_dia', 'min_volume_hora_dia']])

In [154]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true))

In [159]:
mean_absolute_percentage_error(values_test, y_pred)

0.14217285097496668

In [151]:
sqrt(mean_squared_error(y_pred, values_test))

8.146041194256258

In [45]:
np.count_nonzero(values_test)

420

In [47]:
df_train_grouped = df_train.groupby(["tollgate_id", "direction"])
df_test_grouped = df_test.groupby(["tollgate_id", "direction"])
result = []
oob = []
for key, train_data in df_train_grouped:
    test_data = df_test_grouped.get_group(key)
    len_train = len(train_data)
    train_data = train_data.append(test_data)[train_data.columns.tolist()]
    #train_data = feature_transform_split(key, train_data)
    train_data.head()

    regressor_cubic = RandomForestRegressor(n_estimators=500, max_features='sqrt', random_state=10, oob_score=True)

    train_data = pd.DataFrame.reset_index(train_data)
    train_data = train_data.drop("index", axis=1)
    y = train_data.ix[:len_train - 1, :]["volume"]
    
    #del tain_data['date']
    #del train_data['time_start']
    #del train['time_window']
    #del train['minute']
    
    x = train_data.ix[:len_train - 1, :]
    x1 = train_data.ix[len_train:, :]
    del x['time_window']
    del x1['time_window']
    del x['minute']
    del x1['minute']
    del x['time_start']
    del x1['time_start']
    del x['date']
    del x1['date']
    regressor_cubic.fit(x, y)
    resultado_obtido = regressor_cubic.predict(x1)

    #df_h = test_data
    #df_h.to_csv("result/result_split_rf_huehue"+"teste"+".csv", index=False)

    test_data["volume"] = resultado_obtido
    result.append(test_data[['tollgate_id', 'time_window', 'direction', 'volume']])

/home/matheuspds/miniconda3/lib/python3.7/site-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()
/home/matheuspds/miniconda3/lib/python3.7/site-packages/ipykernel_launcher.py:23: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


In [48]:
df_train_grouped

In [121]:
df_result = pd.concat(result, axis=0)

In [122]:
df_result.head()

,tollgate_id,time_window,direction,volume
18,1,"[06:00:00,06:20:00)",0,10.580
19,1,"[06:20:00,06:40:00)",0,18.394
20,1,"[06:40:00,07:00:00)",0,29.686
21,1,"[07:00:00,07:20:00)",0,27.840
22,1,"[07:20:00,07:40:00)",0,38.020


In [123]:
df_test.head()

,time_start,volume,direction,tollgate_id,time_window,date,weekday,weekend,hour,day,...,volume_anterior,volume_anterior_2,media_volume,desvio_padrao,0,1,2,3,4,5
18,2016-10-25 06:00:00,10,0,1,"[06:00:00,06:20:00)",2016-10-25,2,0,6,25,...,0.0,0.0,14.285714,6.447591,0,0,0,0,0,0
19,2016-10-25 06:20:00,17,0,1,"[06:20:00,06:40:00)",2016-10-25,2,0,6,25,...,10.0,0.0,22.428571,14.808299,0,0,0,0,0,0
20,2016-10-25 06:40:00,30,0,1,"[06:40:00,07:00:00)",2016-10-25,2,0,6,25,...,17.0,10.0,28.714286,14.941154,0,0,0,0,0,0
21,2016-10-25 07:00:00,25,0,1,"[07:00:00,07:20:00)",2016-10-25,2,0,7,25,...,30.0,17.0,39.285714,26.208868,0,0,0,0,0,0
22,2016-10-25 07:20:00,38,0,1,"[07:20:00,07:40:00)",2016-10-25,2,0,7,25,...,25.0,30.0,45.428571,23.308490,0,0,0,0,0,0


In [124]:
volumes_predict = df_result['volume'].values.astype(int)
volumes_real = df_test['volume'].values

In [125]:
volumes_real

array([ 10,  17,  30,  25,  38,  44,  52,  44,  46,  50,  40,  50,  12,
         9,  25,  31,  32,  45,  38,  36,  44,  49,  45,  46,  14,  14,
        23,  20,  36,  42,  40,  37,  45,  49,  41,  45,   8,  21,  14,
        28,  37,  46,  62,  48,  57,  78,  67,  58,  11,  17,  19,  27,
        37,  24,  60,  48,  52,  48,  62,  48,  18,  25,  30,  49,  40,
        74, 122, 129, 152, 118, 112, 110,  27,  54,  60,  95,  98, 106,
        94,  94,  88,  85,  93,  88,  29,  27,  40,  83,  98,  93,  77,
        66, 100,  78,  83,  69,  32,  27,  39,  74,  88,  99,  72,  59,
        74,  68,  80,  90,  20,  36,  50,  70,  81, 114,  58,  54,  79,
        57,  73,  83,  19,  23,  53,  66,  99,  97,  87,  96, 104,  85,
        93, 110,  23,  42,  40,  34,  53,  61,  91,  77,  84,  77,  79,
        83,   5,   3,   6,   6,  11,  16,  13,  25,  26,  13,  27,  19,
        10,  10,  14,  37,  42,  58,  13,  17,  13,  19,  12,  17,  28,
        48,  92, 129, 131, 139, 125, 123, 124, 126, 141, 118,  2

In [126]:
volumes_predict

array([ 10,  18,  29,  27,  38,  44,  44,  43,  47,  51,  41,  50,  12,
        13,  24,  30,  33,  45,  35,  36,  44,  49,  45,  46,  13,  16,
        24,  24,  35,  42,  36,  37,  46,  50,  42,  46,   8,  20,  18,
        28,  37,  46,  51,  49,  62,  78,  72,  62,  11,  18,  21,  28,
        37,  30,  49,  48,  56,  50,  64,  51,  17,  24,  30,  48,  41,
        71,  64,  92, 117, 104, 100,  96,  21,  42,  51,  78,  84,  94,
        62,  82,  94,  84,  89,  85,  35,  51,  62,  69,  97, 122,  87,
       108,  89, 111, 106, 107,  35,  50,  56,  69,  94, 109,  90,  97,
       101, 109, 110, 108,  35,  46,  52,  64,  90, 104,  84, 101, 101,
        95,  93, 109,  33,  52,  63,  84,  99, 111,  97, 113, 110, 122,
       108, 113,  31,  41,  56,  57,  77,  87,  92,  76,  85, 102,  90,
       110,  37,  42,  56,  65,  73,  76, 102, 116, 116, 105, 105, 118,
        34,  45,  67,  83,  98, 113,  91, 102, 105,  96, 106, 101,  24,
        26,  40,  74,  92,  95,  60,  61,  88,  73,  78,  68,  2

In [127]:
mean_absolute_percentage_error(volumes_real, volumes_predict)

50.773697096489315

In [128]:
sqrt(mean_squared_error(volumes_predict, volumes_real))

34.942197166994845

In [74]:
for shift_num in range(0, 6):
        f2 = lambda x: x.values[shift_num]

        df_train1[str(shift_num)] = df_train1[["tollgate_id", "direction", "volume", "date", "am_pm"]].groupby(
            ["tollgate_id", "direction", "date", "am_pm"]).transform(f2)

In [77]:
df_train1

,time_start,volume,direction,tollgate_id,time_window,date,weekday,weekend,hour,minute,...,volume_anterior,volume_anterior_2,media_volume,desvio_padrao,0,1,2,3,4,5
0,2016-09-19 00:00:00,13,0,1,"[00:00:00,00:20:00)",2016-09-19,1,0,0,0,...,0.0,0.0,11.4,2.190890,13,6,9,10,14,10
1,2016-09-19 00:20:00,6,0,1,"[00:20:00,00:40:00)",2016-09-19,1,0,0,20,...,13.0,0.0,10.0,4.242641,13,6,9,10,14,10
2,2016-09-19 00:40:00,9,0,1,"[00:40:00,01:00:00)",2016-09-19,1,0,0,40,...,6.0,13.0,8.4,2.966479,13,6,9,10,14,10
3,2016-09-19 01:00:00,10,0,1,"[01:00:00,01:20:00)",2016-09-19,1,0,1,0,...,9.0,6.0,5.8,3.033150,13,6,9,10,14,10
4,2016-09-19 01:20:00,14,0,1,"[01:20:00,01:40:00)",2016-09-19,1,0,1,20,...,10.0,9.0,7.2,5.069517,13,6,9,10,14,10
5,2016-09-19 01:40:00,10,0,1,"[01:40:00,02:00:00)",2016-09-19,1,0,1,40,...,14.0,10.0,7.6,1.949359,13,6,9,10,14,10
6,2016-09-19 02:00:00,7,0,1,"[02:00:00,02:20:00)",2016-09-19,1,0,2,0,...,10.0,14.0,6.0,1.581139,13,6,9,10,14,10
7,2016-09-19 02:20:00,10,0,1,"[02:20:00,02:40:00)",2016-09-19,1,0,2,20,...,7.0,10.0,7.4,2.880972,13,6,9,10,14,10
8,2016-09-19 02:40:00,6,0,1,"[02:40:00,03:00:00)",2016-09-19,1,0,2,40,...,10.0,7.0,7.8,2.774887,13,6,9,10,14,10
9,2016-09-19 03:00:00,9,0,1,"[03:00:00,03:20:00)",2016-09-19,1,0,3,0,...,6.0,10.0,7.4,1.673320,13,6,9,10,14,10


In [4]:
s = np.random.normal(100, 300, 50)

In [5]:
s

array([ 156.66596046,   45.75092329, -133.4981438 ,   88.24351989,
        443.29637242, -141.81641604,  402.14362832,  163.33813689,
         41.9788619 ,  395.6736158 ,  202.16698113,  134.4933092 ,
        300.10275199,   48.61843468,  213.34746028,  546.73588345,
        139.97008796,  689.017899  ,  149.74895845,  257.92674258,
       -643.64699384,  300.36129196,  229.38927887,  -83.2225127 ,
       -283.1997951 , -324.0177248 ,  -66.40315458,  117.3422809 ,
        118.70441529,  309.0111096 ,  257.07688331,  358.34734194,
        323.93856698,  277.71453537,  273.57697305,  -79.61786243,
        672.30907174,   87.33455268,  692.02134521,  106.71185952,
        637.06410955,  345.40580258,  188.39550715,    7.3407823 ,
        157.62316912, -246.8514166 , -355.22132444,  -56.79260158,
        360.2033036 ,  200.47970286])